# 1. Optimization Algorithm

## 1.1 Mini-batch Gradient Descent

- **Batch GD:**
    - Same as we learned before: process all the training set/ all the batch at the same time.


- **Mini-batch GD:**
    - Process some smaller training sets at the same time, also known as mini-batches
    - New notation: $X^{\{t\}} :(n_{x}, t), \; Y^{\{t\}} : (1, t)$


**Idea:**

for t = 1, ..., 5000 {
    
1. Forward propogation on $X^{\{t\}}$
    - vectorized implementation
2. Compute cost: $J^{\{t\}}$, which is based on the mini-batch $X^{\{t\}}, Y^{\{t\}}$
3. Backward propogation to compute gradients using the cost  
4. Update parameters $W^{[l]}, b^{[l]}$

} 

Running all the iterations here is called "**1 epoch**" (1 pass through all the training set/mini-batches) 


**Understanding Mini-batch GD:**

- **Difference:**

    ![](./imgs/mini-vs-all.jpg)
    
    - The reason is that some mini-batches may work well in current neural networks, while others may not, i.e. some fake training examples.
    

- **Mini-batch Size:**
    - Mini-batch size = m: Batch GD (the entire training set is one batch)
        - Time-consuming. 
    - Mini-batch size = 1: Stochastic GD (every example is its own mini-batch)
        - It won't converge. Eventually it will stay around the global minimum.
        - Lose speedup from vectorization. 
    - In practice: between 1 to m. Not too big/small. 
        - Fastest lerning: 1) vectorization ~ 1,000, 2) make progresss without passsing entire training set. 
    
    ![](./imgs/batch-size.jpg)

    - **How to chose:**
        - if small train set ($m \leq 2000$) : use **batch GD**
        - typical mini-batch size: 64, 128, 256, 512, (some number equal to **the power of 2** )
            - Make sure mini-batch fit in CPU/GPU memory; otherwise, it becomes worse.


## 1.2 Exponentially Weighted Averages (EWA)

**Forlumar:**

$$ V_{t} = \beta \; V_{t-1} + (1 - \beta) \; \theta_{t} $$

- $\beta$ can be considered as the average of several $\frac{1}{1-\beta}$ days. 
    - $\beta = 0.9$ : ~ 10 days average (red line)
        - did show adaptive
    - $\beta = 0.98$ : ~ 50 days average (green line)
        - go smooth because of the larger window size, so it adapts slower and showd a right shift 
    - $\beta = 0.5$ : ~ 2 days average (yellow line) 
        - adaptive faster

    ![](./imgs/ewa-ie.jpg)


**Understanding Exponentially Weighted Averages:**

- Expand the formular: 

    $$ V_{t} = \sum_{i=1}^{t} \; (1 - \beta) \; \beta^{(t - i)} \; V_{i} $$

    - where $(1 - \epsilon)^{\frac{1}{\epsilon}} \approx \frac{1}{e} $, which means the exponential curve will approximate 0 after greater than $\epsilon$. Thus, it feels like only $\epsilon$ days are been averaged while the other days have very small weights. 
        - Let $\epsilon = 1 - \beta$ , now we can understand the example above.


- In practice: 

    ![](./imgs/ewa-formular.jpg)

**Bias Correction in EWA:**



The purple line below shows the bias in EWA which is caused by the initialization step $V_{0} = 0$ .  

![](./imgs/ewa-bias.jpg)


**Solution:**

- Instead of taking $V_{t}$ for the next iteration, taking $\frac{V^{t}}{1 - \beta^{t}} $. 
    - Note that as $t$ goes larger, $1 - \beta^{t} \rightarrow 0 $. Thus, it will finally overlay with the original formular. 

    ![](./imgs/ewa-bias-fix.jpg)


## 1.3 Gradient Algorithms with Momentum

**Goal:**

- Let the gradient descent converge faster. In the counter map, it moves slowly in the vertical direction but faster in the horizaontal direction. 

    ![](./imgs/gdm.jpg)

- How to do it? Introduce momentum into Gradient Descent. Like the example above, use Exponentially Weighted Average to smooth the gradients so that we can use a larger learnign rate and rapidly reach the global minimum. 


**Algorithm:**

- Only need to add two lines before updating the parameters 

    On iteration t: 
    
    Compute gradients on current mini-batch
    
$$ v_{dw} = \beta \; v_{dw} + (1 - \beta) \; dw $$
$$ v_{db} = \beta \; v_{db} + (1 - \beta) \; db $$

$$ W = W - \alpha \; v_{dw} $$
$$ b = b - \alpha \; v_{db} $$

- Hyperparameters: $\beta$

**In practice:**
- Just simply initialize $v_{dw}, v_{db}$ as 0, because only about 10 iterations it will be same.
- Removing $(1-\beta)$ . However, it becomes less intuitive, and just a scaling problem for the previous gradients. Some people use it while others not. 


## 1.4 Root Mean Square Propogation (RMSprop)

**Idea:**

- Try to control the gradient descent between parameters so that it can reduce vertical oscillations while move faster in the horizontal way. 


**Algorithm:**

Initialize: 
$$ S_{dw} = 0, \; S_{db} = 0 $$

One iteration t: 

Compute gradients on current mini-batch

$$ S_{dw} = \beta_{2} \; S_{dw} + (1 - \beta_{2}) \; dW^{2} $$
$$ S_{db} = \beta_{2} \; S_{db} + (1 - \beta_{2}) \; db^{2} $$

$$ W := W - \alpha \; \frac{dW}{\sqrt{S_{dw}}} $$
$$ b := b - \alpha \; \frac{db}{\sqrt{S_{db}}} $$

**Note:**
- Once one parameter is too large, by dividing its rooted gradients can make it smaller. Vice versa.
- The rooted gradients should not be zero, so add a tiny $\epsilon \approx 10^{-8}$ to the demoninator when programming.



## 1.5 Adam Optimization Algorithm

**Idea:**

- The combination of Momentum and RMSprop. The most commonly used optimization algorithm. 
- It's called Adaptive Moment Estimation: the first moment is $\beta_{1}$ while the second moment is $\beta_{2}$. 

**Algorithm:**

- Initialize: 

$$ V_{dw} = 0, \; V_{db} = 0, \; S_{dw} = 0, \; S_{db} = 0 $$

- On iteratin t: 

    - compute $dw, db$ on current min-batch
    
    $$ V_{dw} = \beta_{1} \; V_{dw} + (1 - \beta_{1}) \; dw \tag{1}$$
    $$ V_{db} = \beta_{1} \; V_{db} + (1 - \beta_{1}) \; db \tag{2}$$
    
    $$ S_{dw} = \beta_{2} \; S_{dw} + (1 - \beta_{2}) \; dw^2 \tag{3}$$
    $$ S_{db} = \beta_{2} \; S_{db} + (1 - \beta_{2}) \; db^2 \tag{4}$$
    
    $$ V_{dw}^{corrected} = V_{dw} \; / \; (1 - \beta^{t}_{1}) \tag{5} $$
    $$ V_{db}^{corrected} = V_{db} \; / \; (1 - \beta^{t}_{1}) \tag{6} $$
    
    $$ S_{dw}^{corrected} = S_{dw} \; / \; (1 - \beta^{t}_{2}) \tag{7} $$
    $$ S_{db}^{corrected} = s_{db} \; / \; (1 - \beta^{t}_{2}) \tag{8} $$
    
    $$ W := W - \alpha \; \frac{V_{dw}^{corrected}}{\sqrt{S_{dw}^{corrected}} + \epsilon} \tag{9}$$
    $$ b := b - \alpha \; \frac{V_{db}^{corrected}}{\sqrt{S_{db}^{corrected}} + \epsilon} \tag{10}$$
    
- Hyperparameters needed to tune: 
    
    $$\alpha, \; \beta_{1} = 0.9 \; (\text{default}), \; \beta_{2} = 0.999 \; (\text{default}), \; \epsilon $$ 


**Note:**
- The equation 1 and 2 are based on Momentum while the equation 3 and 4 are from RMSprop. 
- All of them should be corrected to remove bias.
- A tiny number $\epsilon$ is added to prevent from being divided by zero, which is fewly tuned. 



## 1.6 Learning Rate Decay

**Issue:**

- Fixing the learning rate may not converge if $\alpha$ is too large. 
    
![](./imgs/fix-learning-rate.jpg)


**Solution:**
- Gradually reduce the learning rate, also called learning rate decay. 

$$ \alpha = \frac{1}{1 + decay-rate \; * \; epoch-num} \; \alpha_{0} $$

- Hyperparameters: $decay-rate, \; \alpha_{0} $

**Other Leraning Rate Decay Methods:**

![](./imgs/learning-rate-methods.jpg) 


## 1.7 Problem of Local Optima

![](./imgs/local-optima.jpg)

- The left figure is the ideal one, which rarely happen.
    - The local optima is the global minimal. 
- The right figure happens a lot in deep learning, espetially when there are millions of parameters. 
    - Thus, the local optima is not the global minimal but **the saddle point**. 
    

**Local optimal are not a problem. The problem is plateaus.**

![](./imgs/plateaus.jpg)

- because the gradients on the plateaus are very closed to 0, so it makes learning very slow.

-----------

# Quiz

In Exponentially Weighted Averages, $\beta$ is very important. Based on the term $\frac{1}{1-\beta}$, 

- The larger the $\beta$, more days are averaged, so the average goes smooth but less adaptive. 
- The smaller the $\beta$, fewer days are averaged, so the average is more oscillated and adaptive. 



---------------

# Assignments

## 1. Mini-Batch Gradient descent

There are two steps:
- **Shuffle**: Create a shuffled version of the training set (X, Y) as shown below. Each column of X and Y represents a training example. Note that the random shuffling is done synchronously between X and Y. Such that after the shuffling the $i^{th}$ column of X is the example corresponding to the $i^{th}$ label in Y. The shuffling step ensures that examples will be split randomly into different mini-batches. 

    ```python
    # Codes: 
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((1,m))
    ```
    
    ![](./imgs/hw-kiank_shuffle.png)

- **Partition**: Partition the shuffled (X, Y) into mini-batches of size `mini_batch_size` (here 64). Note that the number of training examples is not always divisible by `mini_batch_size`. The last mini batch might be smaller, but you don't need to worry about this. When the final mini-batch is smaller than the full `mini_batch_size`, it will look like this: 

    ![](./imgs/hw-kiank_partition.png)

<font color='blue'>
    
**What you should remember**:
- Shuffling and Partitioning are the two steps required to build mini-batches
- Powers of two are often chosen to be the mini-batch size, e.g., 16, 32, 64, 128.

## 2. Momentum

**Initialize the velocity:**

The velocity, $v$, is a python dictionary that needs to be initialized with arrays of zeros. Its keys are the same as those in the `grads` dictionary, that is:
for $l =1,...,L$:
```python
v["dW" + str(l+1)] = ... #(zeros with the same shape as parameters["W" + str(l+1)])
v["db" + str(l+1)] = ... #(zeros with the same shape as parameters["b" + str(l+1)])
```
**Note** that the iterator l starts at 0 in the for loop while the first parameters are v["dW1"] and v["db1"] (that's a "one" on the superscript). This is why we are shifting l to l+1 in the `for` loop.

**Forlumar:**

$$ \begin{cases}
v_{dW^{[l]}} = \beta v_{dW^{[l]}} + (1 - \beta) dW^{[l]} \\
W^{[l]} = W^{[l]} - \alpha v_{dW^{[l]}}
\end{cases}\tag{1}$$

$$\begin{cases}
v_{db^{[l]}} = \beta v_{db^{[l]}} + (1 - \beta) db^{[l]} \\
b^{[l]} = b^{[l]} - \alpha v_{db^{[l]}} 
\end{cases}\tag{2}$$


**Note** that:
- The velocity is initialized with zeros. So the algorithm will take a few iterations to "build up" velocity and start to take bigger steps.
- If $\beta = 0$, then this just becomes standard gradient descent without momentum. 

**How do you choose $\beta$?**

- The larger the momentum $\beta$ is, the smoother the update because the more we take the past gradients into account. But if $\beta$ is too big, it could also smooth out the updates too much. 
- Common values for $\beta$ range from 0.8 to 0.999. If you don't feel inclined to tune this, $\beta = 0.9$ is often a reasonable default. 
- Tuning the optimal $\beta$ for your model might need trying several values to see what works best in term of reducing the value of the cost function $J$. 

<font color='blue'>

**What you should remember**:
- Momentum takes past gradients into account to smooth out the steps of gradient descent. It can be applied with batch gradient descent, mini-batch gradient descent or stochastic gradient descent.
- You have to tune a momentum hyperparameter $\beta$ and a learning rate $\alpha$.

## 3. Adam

**How does Adam work?**
1. It calculates an exponentially weighted average of past gradients, and stores it in variables $v$ (before bias correction) and $v^{corrected}$ (with bias correction). 
2. It calculates an exponentially weighted average of the squares of the past gradients, and  stores it in variables $s$ (before bias correction) and $s^{corrected}$ (with bias correction). 
3. It updates parameters in a direction based on combining information from "1" and "2".

The update rule is, for $l = 1, ..., L$: 

$$\begin{cases}
v_{dW^{[l]}} = \beta_1 v_{dW^{[l]}} + (1 - \beta_1) \frac{\partial \mathcal{J} }{ \partial W^{[l]} } \\
v^{corrected}_{dW^{[l]}} = \frac{v_{dW^{[l]}}}{1 - (\beta_1)^t} \\
s_{dW^{[l]}} = \beta_2 s_{dW^{[l]}} + (1 - \beta_2) (\frac{\partial \mathcal{J} }{\partial W^{[l]} })^2 \\
s^{corrected}_{dW^{[l]}} = \frac{s_{dW^{[l]}}}{1 - (\beta_2)^t} \\
W^{[l]} = W^{[l]} - \alpha \frac{v^{corrected}_{dW^{[l]}}}{\sqrt{s^{corrected}_{dW^{[l]}}} + \varepsilon}
\end{cases}$$
where:
- t counts the number of steps taken of Adam 
- L is the number of layers
- $\beta_1$ and $\beta_2$ are hyperparameters that control the two exponentially weighted averages. 
- $\alpha$ is the learning rate
- $\varepsilon$ is a very small number to avoid dividing by zero


## 4. Compare Three Optimization Methods

**Grandient Descent:**

![](./imgs/hw2-gd.png)

![](./imgs/hw2-gd-res.png)


**Grandient Descent w/ Momentum:**

![](./imgs/hw2-gdm.png)

![](./imgs/hw2-gdm-res.png)


**Adam:**

![](./imgs/hw2-adam.png)

![](./imgs/hw2-adam-res.png)

**Accuracy:**

![](./imgs/hw2-res.jpg)